# Build GPT from scratch

In [15]:
MANUAL_SEED = 1337

## Preprocess dataset

In [1]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [2]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# Check all the unique characters in dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print("Vocabulary size: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size:  65


In [4]:
# Create a mapping of characters to integers
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for i,c in enumerate(chars)}
encode = lambda msg: [stoi[c] for c in msg]               # encode message
decode = lambda e_msg: "".join([itos[i] for i in e_msg])  # decode encoded message

print(encode("Hello, world!"))
print(decode(encode("Hello, world!")))

[20, 43, 50, 50, 53, 6, 1, 61, 53, 56, 50, 42, 2]
Hello, world!


## Tokenize the entire dataset

In [6]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

/home/barbara/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


## Split dataset

In [7]:
n = int(0.9 *len(data))
train_data = data[:n]
val_data = data[n:]

## Get chunck of data for training

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is: {target}")


When input is tensor([18]) the target is: 47
When input is tensor([18, 47]) the target is: 56
When input is tensor([18, 47, 56]) the target is: 57
When input is tensor([18, 47, 56, 57]) the target is: 58
When input is tensor([18, 47, 56, 57, 58]) the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [13]:
torch.manual_seed(MANUAL_SEED)
batch_size = 4  # number of independent sequences to train on in parallel
block_size = 8  # lmaximum context length for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # choose random starting points for each sequence in the batch
    x = torch.stack([data[i:i+block_size] for i in ix])  
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [14]:
xb, yb = get_batch('train')
print("inputs:")
print(xb)
print("targets:")
print(yb)


inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


## Bigram language model

- B: Batch_size
- T: Length of the input sequence
- C: Number of features per token (size of the embedding)

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(MANUAL_SEED)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # Initialize a matrix of shape (vocab_size, vocab_size) with  random values
        # that will be optimized during training
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indeces in the current context
        # the goal is to get (B, T+1), (B, T+2), ... (B, T+max_new_tokens )

        for _ in range(max_new_tokens):
            # get the predictions 
            logits, loss = self(idx)   
            
            # focus only on the last time step (the logit of the last token in the context)
            logits = logits[:, -1, :] # becomes (B, C)

            # softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1)
        
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss.item())

idx = torch.zeros((1,1), dtype=torch.long) # Contains the character corresponding to index 0 (newline)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist())) # decode the first batch of the generated completions


torch.Size([32, 65])
4.878634929656982

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


### Samples from model

- Generation on initialization (loss 4.8786): "SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ"

- Generation after ~20000 steps of trainign (loss 2.5770): "Fouthe f Goversthy harmarend t:
Musthee aved tef t thaphapayeeraie ce. t, ndedigetlot;
W:


A ityove
"

## Train the Model

In [26]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [34]:
batch_size = 32
for steps in range(10000):

    # sample batch
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.576992988586426


In [41]:
print(decode(model.generate(idx, max_new_tokens=200)[0].tolist())) # decode the first batch of the generated completions



hintheth miouro oulof I and s l SS:
Iffof; hat t-asoresere ashath fover;

AUMENGHave ie? nds to wisus, athal
Fiotha her owa
Fouidif toury aris dor yoress ane hit in,
O:
LETAUns.
Isat t ust far thas s 
